Please note I'm in a python notebook, making use of magic commands (e.g. `%sql`) to drop into different language blocks

In [2]:
# Create test dataframes and register them in spark sql context as tables

mock_data1 = [('Bob', 1),('Sue', 3), ('Paul', 4),('Alice', 2),('Josh', 2)]
spark.createDataFrame(mock_data1, ['colA', 'colB']).registerTempTable('t1')

mock_data2 = [('Bob', 3,'XYZ'),('Sue', 2,'XYZ'), ('Pam', 2,'XYZ'),('Arthur', 2,'XYZ'), ('Josh',1,'ABC')]
spark.createDataFrame(mock_data2, ['colA', 'colB', 'colC']).registerTempTable('t2')


In [3]:
%sql

update t1
set t1.colB=CASE WHEN t2.colB>t1.colB THEN t2.colB ELSE t1.colB +t2.colB END
from t1
inner join t2 ON t1.colA=t2.colA
where t2.colC='XYZ'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input 'from' expecting (line 3, pos 0)

== SQL ==
update t1
set t1.colB=CASE WHEN t2.colB>t1.colB THEN t2.colB ELSE t1.colB +t2.colB END
from t1
^^^
inner join t2 ON t1.colA=t2.colA
where t2.colC='XYZ'

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:53)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:69)
	at com.databricks.sql.parser.DatabricksSqlParser$$anonfun$parsePlan$1.apply(DatabricksSqlParser.scala:64)
	at com.databricks.sql.parser.DatabricksSqlParser$$anonfun$parsePlan$1.apply(DatabricksSqlParser.scala:61)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:84)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:61)
	at org.apache.spark.sql.SparkSession$$anonfun$6.apply(SparkSession.scala:694)
	at org.apache.spark.sql.SparkSession$$anonfun$6.apply(SparkSession.scala:694)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:693)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:716)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:88)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:34)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:34)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:141)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:381)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:358)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:272)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:49)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:358)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at scala.util.Try$.apply(Try.scala:192)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:639)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:485)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:597)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:390)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
	at java.lang.Thread.run(Thread.java:748)

	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDrive

First instinct, use a subquery? ...

In [5]:
%sql

update t1
set t1.colB=
(select CASE WHEN t2.colB>t1.colB THEN t2.colB ELSE t1.colB +t2.colB END
from t1
inner join t2 ON t1.colA=t2.colA
where t2.colC='XYZ')

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(LogicalRDD [colA#10228, colB#10229], false
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:166)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:56)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:50)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:77)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply0(PreprocessTableUpdateEdge.scala:50)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:46)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:39)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:115)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply

Nope, doesn't work. But does remind me that updates are for delta sources only. Let's convert to delta tables

In [7]:
%sql 
drop table if exists dt1;
drop table if exists dt2;

CREATE TABLE dt1
USING delta
AS SELECT * from t1;

CREATE TABLE dt2
USING delta
AS SELECT * from t2;

In [8]:
%sql

update dt1
set dt1.colB=
(select CASE WHEN dt2.colB>dt1.colB THEN dt2.colB ELSE dt1.colB +dt2.colB END
from dt1
inner join dt2 ON dt1.colA=dt2.colA
where dt2.colC='XYZ')

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:201)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge$$anonfun$writeFiles$1$$anonfun$apply$1.apply(TransactionalWriteEdge.scala:147)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge$$anonfun$writeFiles$1$$anonfun$apply$1.apply(TransactionalWriteEdge.scala:130)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge$$anonfun$writeFiles$1.apply(TransactionalWriteEdge.scala:130)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge$$anonfun$writeFiles$1.apply(TransactionalWriteEdge.scala:87)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:405)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:272)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:386)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger$class.recordOperation(UsageLogger.scala:67)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging$class.recordOperation(UsageLogger.scala:342)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.recordOperation(OptimisticTransaction.scala:80)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging$class.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.recordDeltaOperation(OptimisticTransaction.scala:80)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge$class.writeFiles(TransactionalWriteEdge.scala:87)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.writeFiles(OptimisticTransaction.scala:80)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWrite$class.writeFiles(TransactionalWrite.scala:103)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.writeFiles(OptimisticTransaction.scala:80)
	at com.databricks.sql.transaction.tahoe.commands.UpdateCommandEdge.com$databricks$sql$transaction$tahoe$commands$UpdateCommandEdge$$rewriteFiles(UpdateCommandEdge.scala:233)
	at com.databricks.sql.transaction.tahoe.commands.UpdateCommandEdge.com$databricks$sql$transaction$tahoe$commands$UpdateCommandEdge$$performUpdate(UpdateCommandEdge.scala:109)
	at com.databricks.sql.transaction.tahoe.commands.UpdateCommandEdge$$anonfun$run$1$$anonfun$apply$1$$anonfun$apply$mcV$sp$1.apply(UpdateCommandEdge.scala:66)
	at com.databricks.sql.transaction.tahoe.commands.UpdateCommandEdge$$anonfun$run$1$$anonfun$apply$1$$anonfun$apply$mcV$sp$1.apply(UpdateCommandEdge.scala:65)
	at com.databricks.sql.transac

Issue with join. Let's try using the Merge statement. This seems like the best solution here, assuming we're levering the delta architecture and storage format

<https://docs.databricks.com/spark/latest/spark-sql/language-manual/merge-into.html>

In [10]:
%sql 
merge into dt1
using dt2 ON dt1.colA=dt2.colA and dt2.colB > dt1.colB and dt2.colC = 'XYZ'
WHEN MATCHED THEN UPDATE set colB = dt2.colB;

merge into dt1
using dt2 ON dt1.colA=dt2.colA and dt2.colB <= dt1.colB and dt2.colC = 'XYZ'
WHEN MATCHED THEN UPDATE set colB = dt1.colB + dt2.colB;

This worked, but the sequential updates made colB be updated accidentally in the 2nd statement

In [12]:
%sql 
drop view if exists joined;

create temporary view joined as
select dt1.colA, CASE WHEN dt2.colB>dt1.colB THEN dt2.colB ELSE dt1.colB + dt2.colB END as colB
from dt1 inner join dt2 ON dt1.colA=dt2.colA
where dt2.colC='XYZ';

merge into dt1
using joined ON dt1.colA=joined.colA
WHEN MATCHED THEN UPDATE set colB = joined.colB;

Success!

In [14]:
%sql
select * from dt1

colA,colB
Alice,2
Josh,2
Paul,4
Bob,3
Sue,5
